<a href="https://colab.research.google.com/github/SeungukJeong/ML4AI/blob/main/catboost_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
project_path = '/content/drive/MyDrive/FinalProject'

os.chdir(project_path)
os.getcwd()

'/content/drive/.shortcut-targets-by-id/1P8ce0awz1gepxsBj5WPzoWZSy8UT7Yku/FinalProject'

Library  불러오기...(추가 예정)

In [ ]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
import os 

import datetime

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv(project_path + "/Feature_data/df_merge.csv")

In [ ]:
train.head()

,segment_id,time_to_eruption,s1_A_pow,s1_A_num,s1_BH_pow,s1_BH_num,s1_BL_pow,s1_BL_num,s1_C_pow,s1_C_num,s1_D_pow,s1_D_num,s2_A_pow,s2_A_num,s2_BH_pow,s2_BH_num,s2_BL_pow,s2_BL_num,s2_C_pow,s2_C_num,s2_D_pow,s2_D_num,s3_A_pow,s3_A_num,s3_BH_pow,s3_BH_num,s3_BL_pow,s3_BL_num,s3_C_pow,s3_C_num,s3_D_pow,s3_D_num,s4_A_pow,s4_A_num,s4_BH_pow,s4_BH_num,s4_BL_pow,s4_BL_num,s4_C_pow,s4_C_num,s4_D_pow,s4_D_num,s5_A_pow,s5_A_num,s5_BH_pow,s5_BH_num,s5_BL_pow,s5_BL_num,s5_C_pow,s5_C_num,s5_D_pow,s5_D_num,s6_A_pow,s6_A_num,s6_BH_pow,s6_BH_num,s6_BL_pow,s6_BL_num,s6_C_pow,s6_C_num,s6_D_pow,s6_D_num,s7_A_pow,s7_A_num,s7_BH_pow,s7_BH_num,s7_BL_pow,s7_BL_num,s7_C_pow,s7_C_num,s7_D_pow,s7_D_num,s8_A_pow,s8_A_num,s8_BH_pow,s8_BH_num,s8_BL_pow,s8_BL_num,s8_C_pow,s8_C_num,s8_D_pow,s8_D_num,s9_A_pow,s9_A_num,s9_BH_pow,s9_BH_num,s9_BL_pow,s9_BL_num,s9_C_pow,s9_C_num,s9_D_pow,s9_D_num,s10_A_pow,s10_A_num,s10_BH_pow,s10_BH_num,s10_BL_pow,s10_BL_num,s10_C_pow,s10_C_num,s10_D_pow,s10_D_num,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_25%,sensor_1_50%,sensor_1_75%,sensor_1_max,sensor_1_skew,sensor_1_mad,sensor_1_kurtosis,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_25%,sensor_2_50%,sensor_2_75%,sensor_2_max,sensor_2_skew,sensor_2_mad,sensor_2_kurtosis,sensor_3_mean,sensor_3_std,sensor_3_min,sensor_3_25%,sensor_3_50%,sensor_3_75%,sensor_3_max,sensor_3_skew,sensor_3_mad,sensor_3_kurtosis,sensor_4_mean,sensor_4_std,sensor_4_min,sensor_4_25%,sensor_4_50%,sensor_4_75%,sensor_4_max,sensor_4_skew,sensor_4_mad,sensor_4_kurtosis,sensor_5_mean,sensor_5_std,sensor_5_min,sensor_5_25%,sensor_5_50%,sensor_5_75%,sensor_5_max,sensor_5_skew,sensor_5_mad,sensor_5_kurtosis,sensor_6_mean,sensor_6_std,sensor_6_min,sensor_6_25%,sensor_6_50%,sensor_6_75%,sensor_6_max,sensor_6_skew,sensor_6_mad,sensor_6_kurtosis,sensor_7_mean,sensor_7_std,sensor_7_min,sensor_7_25%,sensor_7_50%,sensor_7_75%,sensor_7_max,sensor_7_skew,sensor_7_mad,sensor_7_kurtosis,sensor_8_mean,sensor_8_std,sensor_8_min,sensor_8_25%,sensor_8_50%,sensor_8_75%,sensor_8_max,sensor_8_skew,sensor_8_mad,sensor_8_kurtosis,sensor_9_mean,sensor_9_std,sensor_9_min,sensor_9_25%,sensor_9_50%,sensor_9_75%,sensor_9_max,sensor_9_skew,sensor_9_mad,sensor_9_kurtosis,sensor_10_mean,sensor_10_std,sensor_10_min,sensor_10_25%,sensor_10_50%,sensor_10_75%,sensor_10_max,sensor_10_skew,sensor_10_mad,sensor_10_kurtosis,sensor1_entropy,sensor2_entropy,sensor3_entropy,sensor4_entropy,sensor5_entropy,sensor6_entropy,sensor7_entropy,sensor8_entropy,sensor9_entropy,sensor10_entropy
0,1136037770,12262005,0.000000,0.0,5630.045792,271.0,46725.41373,841.0,37140.92555,452.0,72426.03253,1848.0,30798.67330,421.0,331692.5490,2631.0,45379.76338,468.0,22777.93270,225.0,206242.4745,1657.0,16604.279570,522.0,47417.80271,1311.0,40676.07538,769.0,33720.22967,437.0,80642.15558,1777.0,8226.754250,220.0,182647.14050,2530.0,35439.62087,582.0,24749.72100,356.0,103910.18220,1651.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9850.349498,214.0,16342.737760,354.0,36798.89383,596.0,77124.06073,455.0,66246.42233,1193.0,64934.954960,1035.0,102382.207500,1654.0,9591.874414,200.0,54211.46409,436.0,54659.34581,936.0,55699.430380,871.0,171324.95660,2096.0,7302.118775,131.0,52497.48299,420.0,56240.59989,850.0,40650.624080,635.0,83811.91721,1731.0,48613.30583,738.0,27587.75068,383.0,136269.7643,1950.0,2120.558313,43.0,81355.19078,1444.0,84886.27047,807.0,60996.87159,442.0,157529.5636,1825.0,-1.610323,303.096099,-1421,-199,0,199,1353,-0.057494,238.836278,0.247736,4.613823,678.330828,-5199,-391,0,403,5572,-0.003121,508.512310,2.512873,3.559724,290.872517,-1453,-184,0,187,1413,0.071352,226.776061,0.590296,2.019983,384.140813,-1960,-254,0,258,1699,0.019599,305.079370,0.103267,0.000000,0.000000,0,0,0,0,0,0.000000,0.000000,0.000000,-9.718855,696.936360,-2776,-481,0,475,2432,-0.100291,559.265248,-0.105076,7.060032,521.109527,-2338,-344,0,352,2969,0.067045,413.105345,0.194853,1.268362,620.950335,-2878,-423,0,426,2544,-0.007263,497.883551,-0.060766,1.465042,372.386747,-1747,-24

In [ ]:
train.isnull().sum()

segment_id            0
time_to_eruption      0
s1_A_pow             28
s1_A_num             28
s1_BH_pow            28
                   ... 
sensor6_entropy       0
sensor7_entropy      42
sensor8_entropy     440
sensor9_entropy     124
sensor10_entropy     24
Length: 212, dtype: int64

In [ ]:
df=train.fillna(0)
df.isnull().sum()

segment_id          0
time_to_eruption    0
s1_A_pow            0
s1_A_num            0
s1_BH_pow           0
                   ..
sensor6_entropy     0
sensor7_entropy     0
sensor8_entropy     0
sensor9_entropy     0
sensor10_entropy    0
Length: 212, dtype: int64

In [ ]:
df2=df.corr()

In [ ]:
df_corr = df2.fillna(0)

Why not PCA???

In [ ]:
# sns.clustermap(df_corr, 
#                annot = True,      # 실제 값 화면에 나타내기
#                cmap = 'RdYlBu_r',  # Red, Yellow, Blue 색상으로 표시
#                vmin = -1, vmax = 1, #컬러차트 -1 ~ 1 범위로 표시
#               )

Modeling

In [ ]:
X = df.drop(['segment_id','time_to_eruption'],axis=1)
y = df['time_to_eruption']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True, random_state=10)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, shuffle=True, random_state=10)

Modeling에 앞서 train, test set을 StandardScaler로 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
model = CatBoostRegressor(iterations=102,
                           depth=4,
                           learning_rate=0.001,
                           loss_function='MAE',
                           verbose=True)

In [ ]:
train_dataset = Pool(data=X_train,
                     label=y_train,
                     )
    
eval_dataset = Pool(data=X_val,
                    label=y_val,
                    )

In [ ]:
model.fit(train_dataset,
          use_best_model=True,
          verbose = 0,
          eval_set=eval_dataset)

In [ ]:
y_pred = model.predict(Pool(data=X_test))

In [ ]:
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

MAE: 11488570.82238302


HyperParameter tuning by Optuna

In [ ]:
!pip3 install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 7.0 MB/s 
     |████████████████████████████████| 209 kB 31.2 MB/s 
     |████████████████████████████████| 81 kB 7.3 MB/s 
     |████████████████████████████████| 78 kB 5.3 MB/s 
     |████████████████████████████████| 50 kB 6.5 MB/s 
     |████████████████████████████████| 147 kB 58.1 MB/s 
     |████████████████████████████████| 112 kB 20.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=154aeeffff198f15dab5cea3967774f2bf1717a7522184e73c01f8fc84afc2f3
  Stored in directory: /root/.cache/pip/wheels/7f/1a/65/84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built pyperclip


In [ ]:
import optuna
from optuna.samplers import TPESampler

In [ ]:
def objective(trial):
    model = CatBoostRegressor(
        iterations=trial.suggest_int("iterations", 2, 1000),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1, log=True),
        depth=trial.suggest_int("depth", 2, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 50),
        verbose=True
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return mean_absolute_error(y_test, y_pred)

In [ ]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="catboost", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
602:	learn: 6989229.3187404	total: 34.9s	remaining: 20.8s
603:	learn: 6984894.4292369	total: 35s	remaining: 20.8s
604:	learn: 6982000.1285273	total: 35s	remaining: 20.7s
605:	learn: 6976657.2673418	total: 35.1s	remaining: 20.7s
606:	learn: 6974431.0846373	total: 35.1s	remaining: 20.6s
607:	learn: 6971615.6784450	total: 35.2s	remaining: 20.5s
608:	learn: 6965592.5367216	total: 35.2s	remaining: 20.5s
609:	learn: 6961295.8937698	total: 35.3s	remaining: 20.4s
610:	learn: 6957235.7973260	total: 35.3s	remaining: 20.4s
611:	learn: 6952997.0654684	total: 35.4s	remaining: 20.3s
612:	learn: 6949814.9162037	total: 35.5s	remaining: 20.2s
613:	learn: 6946811.7458361	total: 35.5s	remaining: 20.2s
614:	learn: 6944392.8930257	total: 35.6s	remaining: 20.1s
615:	learn: 6940271.5664107	total: 35.6s	remaining: 20.1s
616:	learn: 6936537.7934027	total: 35.7s	remaining: 20s
617:	learn: 6932509.2845268	total: 35.7s	remaining: 20s
618:	learn: 6927854.6872834	total: 35.8s	rem

In [ ]:
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  100
Best trial:
  Value:  11853077.949646749
  Params: 
    iterations: 2
    learning_rate: 0.0012379758648138576
    depth: 2
    l2_leaf_reg: 0.1403867003917341
    bootstrap_type: Bayesian
    random_strength: 2.2506494324144737
    bagging_temperature: 9.952937479501418
    od_type: Iter
    od_wait: 22


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, shuffle=True, random_state=10)

In [ ]:
n_fold = 5
cv = KFold(n_splits=n_fold, shuffle=True, random_state=10)
prediction = np.zeros(len(test))
mape_, mae, rmse = [], [], []

params = {
            'iterations':1000,
            'learning_rate':0.1,
            'depth':6,
            'eval_metric':'RMSE'
}

for fold, (train_index, val_index) in enumerate(cv.split(X)):
    X_train = X.iloc[train_index,:]
    X_val = X.iloc[val_index,:]

    y_train = y.iloc[train_index]
    y_val = y.iloc[val_index]
          
    clf = CatBoostRegressor(**params)  
    
    train_dataset = Pool(data=X_train,
                     label=y_train,
                     )
    
    eval_dataset = Pool(data=X_val,
                    label=y_val,
                    )
    
    clf.fit(train_dataset,
              use_best_model=True,
              verbose = 0,
              eval_set=eval_dataset)
   
    y_pred = clf.predict(Pool(data=X_test))
    
    mape_.append(mape(y_test, y_pred))
    mae.append(mean_absolute_error(y_test, y_pred))
    rmse.append(np.sqrt(mean_squared_error(y_test, y_pred)))

    print(f"fold: {fold}, MAPE: {mape(y_test, y_pred)}")
    print(f"fold: {fold}, MAE: {mean_absolute_error(y_test, y_pred)}")
    print(f"fold: {fold}, RMSE: {np.sqrt(mean_squared_error(y_test, y_pred))}")

    # test array predictions
    prediction += clf.predict(Pool(data=test))
        
prediction /= n_fold

print('CV mean MAPE:  {0:.4f}, std: {1:.4f}.'.format(np.mean(mape_), np.std(mape_)))
print('CV mean MAE: {0:.4f}, std: {1:.4f}.'.format(np.mean(mae), np.std(mae)))
print('CV mean RMSE: {0:.4f}, std: {1:.4f}.'.format(np.mean(rmse), np.std(rmse)))